# Datendiskreption


!pip install socceraction xgboost matplotsoccer pandas tqdm

In [1]:
import pandas as pd
import socceraction.spadl as spadl
import warnings
import numpy as np
import matplotlib.pyplot as plt
from pygam import LogisticGAM, s, f

In [2]:
# Ignore warnings
warnings.filterwarnings(
  action="ignore", 
  message="credentials were not supplied. open data access only"
)
warnings.simplefilter(
  action='ignore', 
  category=pd.errors.PerformanceWarning
)
pd.set_option('future.no_silent_downcasting', True)

warnings.filterwarnings(
    "ignore",
    message="A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method."
)

In [3]:
# 1. Lade den Datensatz
spadl_h5 = ".\\xSuccess\\top5_15-16_spadl\\match_data.h5"

with pd.HDFStore(spadl_h5) as spadlstore:
    # Spiele laden
    games = spadlstore["games"]
    
    # Du kannst auch teams / players laden, wenn du sie für später brauchst
    teams = spadlstore["teams"]
    players = spadlstore["players"]
    
    actions_list = []
    
    # Über alle Spiele iterieren
    for gid in games.game_id:
        df_actions = spadlstore[f"actions/game_{gid}"]
        # Ggf. noch die 'game_id' explizit als Spalte hinzufügen,
        # falls du sie später brauchst.
        df_actions["game_id"] = gid
        
        actions_list.append(df_actions)
    
    # Zu einem großen DataFrame zusammenfügen
    actions = pd.concat(actions_list, ignore_index=True)

FileNotFoundError: ``D:\Programmierung\GitHub\phd_yannik\src\Projekte\xSuccess\top5_15-16_spadl`` does not exist

Zusammenführen von DataFrames

In [ ]:
actions = (
    actions
    .merge(spadl.actiontypes_df(), how="left")
    .merge(spadl.results_df(), how="left")
    .merge(spadl.bodyparts_df(), how="left")
)
actions.head()

,game_id,original_event_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,type_id,result_id,bodypart_id,action_id,type_name,result_name,bodypart_name
0,3890561,644e16d7-10ca-45f0-8128-fc0055d6f753,1,0.482,175,8387.0,52.9375,34.340,52.2375,36.210,0,1,4,0,pass,success,foot_left
1,3890561,329a1879-2521-4614-8c68-b4798b0e5d23,1,0.935,175,5460.0,52.2375,36.210,51.5375,35.870,21,1,0,1,dribble,success,foot
2,3890561,77e2ddaf-6de3-49e7-a318-7d765799b543,1,1.015,175,5460.0,51.5375,35.870,46.7250,32.640,0,1,4,2,pass,success,foot_left
3,3890561,1b91a029-f722-4b0d-b9d5-53cdc776f9e3,1,2.167,175,6039.0,46.7250,32.640,45.3250,30.005,21,1,0,3,dribble,success,foot
4,3890561,2c51f271-c812-45af-896b-06f49a14a5bb,1,2.954,175,6039.0,45.3250,30.005,29.5750,15.725,0,1,5,4,pass,success,foot_right


Definieren was Torerfolg ist

In [ ]:
goals_mask = (actions["type_name"] == "shot") & (actions["result_name"] == "success")
goals = actions[goals_mask]

Test: Nur direkte Schüsse die zu Toren führten

In [ ]:
shots = actions[actions["type_name"] == "shot"]

print(shots["result_name"].value_counts())

# Optional: Groupby pro Typ (hier macht "shot" allein Sinn),
# aber verallgemeinert:
shot_success_counts = shots.groupby(["type_name","result_name"]).size().reset_index(name="count")
shot_success_counts


result_name
fail       38900
success     4209
Name: count, dtype: int64


,type_name,result_name,count
0,shot,fail,38900
1,shot,success,4209


In [ ]:
type_id_21 = actions[actions["type_id"] == 22]

In [ ]:
type_id_21_counts = type_id_21.groupby(["type_name","type_id"]).size().reset_index(name="count")
type_id_21_counts

In [15]:
# Gruppiere alle Aktionen nach Aktionsart und Ergebnis
success_distribution = actions.groupby(["type_name", "result_name"]).size().reset_index(name="count")

# Ausgabe anzeigen
print(success_distribution)


           type_name  result_name    count
0          bad_touch         fail    53730
1          bad_touch      owngoal      157
2          clearance      success    87459
3     corner_crossed         fail     9762
4     corner_crossed      success     4553
5       corner_short         fail      430
6       corner_short      success     3183
7              cross         fail    28557
8              cross      offside      308
9              cross      success    11580
10           dribble      success  1422859
11              foul         fail    48472
12              foul     red_card      163
13              foul  yellow_card     6724
14  freekick_crossed         fail    15786
15  freekick_crossed      offside      458
16  freekick_crossed      success    10044
17    freekick_short         fail      485
18    freekick_short      offside       21
19    freekick_short      success    26803
20          goalkick         fail    13102
21          goalkick      offside       16
22         

In [14]:
dribbles = actions[actions["type_name"] == "dribble"]

print(dribbles["result_name"].value_counts())

# Optional: Groupby pro Typ (hier macht "dribbles" allein Sinn),
# aber verallgemeinert:
dribbles_success_counts = dribbles.groupby(["type_name","result_name"]).size().reset_index(name="count")
dribbles_success_counts

result_name
success    1422859
Name: count, dtype: int64


,type_name,result_name,count
0,dribble,success,1422859


In [ ]:
def previous_action_type_of_goals(df_actions):
    # df_actions: alle Aktionen eines Spiels
    # 1) Filtern nach erfolgreichen Toren (shots)
    goals_mask = (df_actions["type_name"] == "shot") & (df_actions["result_name"] == "success")
    goals_only = df_actions[goals_mask]
    
    # 2) Für jeden Treffer hole die vorherige Aktion (im selben Spiel)
    prev_action_ids = goals_only["action_id"] - 1
    
    # 3) Merge / join mit den eigentlichen Aktionen
    prev_actions = df_actions[df_actions["action_id"].isin(prev_action_ids)]
    
    # 4) groupby und zählen
    result = prev_actions.groupby("type_name").size().reset_index(name="count")
    return result.sort_values("count", ascending=False)

# Im Loop über alle Spiele anwenden oder für ein Beispielspiel:
counts_for_game = previous_action_type_of_goals(actions)
print(counts_for_game)


Für alle Spiele

In [ ]:
results = []
for gid in games.game_id:
    actions_g = pd.read_hdf(spadl_h5, f"actions/game_{gid}")
    actions_g = (
        actions_g
        .merge(spadl.actiontypes_df(), how="left")
        .merge(spadl.results_df(), how="left")
        # ...
    )
    prev_counts = previous_action_type_of_goals(actions_g)
    # prev_counts enthält z. B. Spalten [type_name, count]
    # Du kannst auch eine neue Spalte 'game_id' anhängen, falls du pro Spiel auswerten willst
    prev_counts["game_id"] = gid
    results.append(prev_counts)

df_all = pd.concat(results, ignore_index=True)

# Jetzt gruppieren wir noch mal über alle Spiele:
final_res = df_all.groupby("type_name")["count"].sum().reset_index()
final_res = final_res.sort_values("count", ascending=False)
print(final_res)

In [ ]:


plt.bar(final_res["type_name"], final_res["count"])
plt.xticks(rotation=90)
plt.title("Aktionstypen unmittelbar vor Torerfolg (Anzahl)")
plt.xlabel("Aktionstyp")
plt.ylabel("Häufigkeit")
plt.show()

DF für alle Spiele erstellen

In [ ]:
with pd.HDFStore(spadl_h5) as spadlstore:
    # Spiele laden
    games = spadlstore["games"]
    
    # Du kannst auch teams / players laden, wenn du sie für später brauchst
    teams = spadlstore["teams"]
    players = spadlstore["players"]
    
    all_actions_list = []
    
    # Über alle Spiele iterieren
    for gid in games.game_id:
        df_actions = spadlstore[f"actions/game_{gid}"]
        # Ggf. noch die 'game_id' explizit als Spalte hinzufügen,
        # falls du sie später brauchst.
        df_actions["game_id"] = gid
        
        all_actions_list.append(df_actions)
    
    # Zu einem großen DataFrame zusammenfügen
    all_actions = pd.concat(all_actions_list, ignore_index=True)

In [ ]:
all_actions = (
    all_actions
    .merge(spadl.actiontypes_df(), how='left', on='type_id')
    .merge(spadl.results_df(), how='left', on='result_id')
    .merge(spadl.bodyparts_df(), how='left', on='bodypart_id')
)

# Jetzt steht in all_actions alles aus allen Spielen
print(all_actions.shape)
all_actions.head()

Neues Dataframe namens "goal" 

In [ ]:
goals = all_actions[
    (all_actions["type_name"] == "shot") &
    (all_actions["result_name"] == "success")
]

Spielminute berechnen

In [ ]:
def get_match_minute(row):
    # offset = 45 * (period_id-1)
    offset = 45 * (row["period_id"] - 1)
    minute = int(offset + row["time_seconds"] // 60)
    return minute

In [ ]:
goals["minute"] = goals.apply(get_match_minute, axis=1)

Phasen definieren in denen Tore fallen können
Beispiel:
1. und 2. Halbzeit
2. alle 15 Minuten
3. alle 5 minuten

In [ ]:
bins = [0,15,30,45,60,75,90,120]  # 120, falls du Verlängerung bis Minute ~120 abdecken möchtest
labels = ["0-15","16-30","31-45","46-60","61-75","76-90","91+"]
goals["phase_15min"] = pd.cut(goals["minute"], bins=bins, labels=labels, right=True)


Tore pro 15-Minuten-Phase zählen

In [ ]:
phase_counts = goals.groupby("phase_15min").size().reset_index(name="count")
print(phase_counts)

Visualisierung

In [ ]:
plt.bar(phase_counts["phase_15min"], phase_counts["count"])
plt.xlabel("Spielphase (Minuten)")
plt.ylabel("Anzahl der Tore")
plt.title("Tore pro 15-Minuten-Intervall")
plt.xticks(rotation=45)
plt.show()


Aufteilen der Daten zwischen Standards und Laufendes Spiel

In [ ]:
standard_types = [
    "free_kick_crossed", 
    "free_kick_short",
    "corner_crossed",
    "corner_short",
    "throw_in",
    "goal_kick",
    "penalty",
    "kick_off"
]


In [ ]:


all_actions["situation_type"] = np.where(
    all_actions["type_name"].isin(standard_types),
    "standard",
    "open_play"
)

In [ ]:
df_standard_and_open = all_actions.copy()

# Jetzt enthält df_standard_and_open jede Aktion,
# und in der Spalte "situation_type" steht entweder "standard" oder "open_play".


In [ ]:
df_standard_and_open[["game_id", "type_name", "result_name", "situation_type"]].head(20)

Nur Standards

In [ ]:
df_standards = df_standard_and_open[df_standard_and_open["situation_type"] == "standard"]
#df_standards.count

Erfolgreiche vs erfolglose Standardsituationen

In [ ]:
df_standards_success = df_standards[df_standards["result_name"] == "success"]
df_standards_fail = df_standards[df_standards["result_name"] != "success"]

Nur Open Play

In [ ]:
df_openplay = df_standard_and_open[df_standard_and_open["situation_type"] == "open_play"]
#df_openplay.count

Erfolgreiche vs. erfolglose Open Play Aktionen vergleichen

In [ ]:
df_openplay_success = df_openplay[df_openplay["result_name"] == "success"]
df_openplay_fail = df_openplay[df_openplay["result_name"] != "success"]

# GAM ausprobieren

In [ ]:
# Angenommen, all_actions enthält die Ereignisdaten
# Filter nur Shots
shots_df = all_actions[all_actions["type_name"] == "shot"].copy()

# Zielvariable: Tor oder nicht
shots_df["is_goal"] = np.where(shots_df["result_name"] == "success", 1, 0)

# Beispiel: dist = Entfernung zum Tor
# (Koordinaten reichen von 0..105 in x und 0..68 in y, in StatsBomb-ähnlichen Daten)
# Tor liegt meist bei x=105, y=34, also:
shots_df["dist_to_goal"] = np.sqrt((105 - shots_df["start_x"])**2 + (34 - shots_df["start_y"])**2)
shots_df["angle"] = np.arctan2((7.32/2), shots_df["dist_to_goal"])  # sehr grobe Approx.

# (Nur als Demo) X-Spalten: Distanz, Winkel, Spielminute
shots_df["minute"] = (shots_df["period_id"] - 1) * 45 + (shots_df["time_seconds"] // 60)

In [ ]:
shots_df = shots_df.dropna(subset=["dist_to_goal", "minute", "angle"])

In [ ]:
# Feature-Matrix X und Ziel Y definieren
X = shots_df[["dist_to_goal", "minute", "angle"]]  # Beispiele für kontinuierliche Variablen
y = shots_df["is_goal"]  # binär (0/1)

# Modell definieren: s(...) sagt pyGAM, dass jede dieser Variablen
# als glatter (Spline-)Term geschätzt wird.
gam = LogisticGAM(
    s(0) +  # s(0) bezieht sich auf die erste Spalte in X => dist_to_goal
    s(1) +  # => minute
    s(2)    # => angle
)

# Modell fitten
gam.fit(X, y)

# Modellgüte etc. anschauen
print("AIC:", gam.statistics_['AIC'])
print("Pseudo R^2:", gam.statistics_['pseudo_r2'])
print("Log-Likelihood:", gam.statistics_['loglikelihood'])


Vorhersagen

In [ ]:
p_goal = gam.predict_proba(X)
shots_df["p_goal_gam"] = p_goal

In [ ]:
from sklearn.metrics import roc_auc_score, brier_score_loss, log_loss

auc = roc_auc_score(y, p_goal)
brier = brier_score_loss(y, p_goal)
ll = log_loss(y, p_goal)

print("AUC:", auc)
print("Brier:", brier)
print("Log Loss:", ll)


# open play vs standards

In [ ]:
shots_df["is_standard"] = np.where(shots_df["situation_type"] == "standard", 1, 0)
X = shots_df[["dist_to_goal", "angle", "is_standard"]]
y = shots_df["is_goal"]  # binär (0/1)

gam = LogisticGAM(
    s(0) +  # dist
    s(1) +  # angle
    f(2)    # factor/kategorisch => hier "is_standard"
).fit(X, y)

In [ ]:
# Modell fitten
gam.fit(X, y)

# Modellgüte etc. anschauen
print("AIC:", gam.statistics_['AIC'])
print("Pseudo R^2:", gam.statistics_['pseudo_r2'])
print("Log-Likelihood:", gam.statistics_['loglikelihood'])

In [ ]:
p_goal = gam.predict_proba(X)
shots_df["p_goal_gam"] = p_goal

In [ ]:
auc = roc_auc_score(y, p_goal)
brier = brier_score_loss(y, p_goal)
ll = log_loss(y, p_goal)

print("AUC:", auc)
print("Brier:", brier)
print("Log Loss:", ll)


Test

In [ ]:
# Kategorische Variable in 0/1 umwandeln:
shots_df["is_standard"] = np.where(shots_df["situation_type"] == "standard", 1, 0)

# X anlegen:
X = shots_df[["dist_to_goal", "angle", "minute", "is_standard"]]
y = shots_df["is_goal"]

# Definieren, dass wir Splines auf dist_to_goal, angle, minute legen
# und 'is_standard' als Faktorterm:
gam = LogisticGAM(
    s(0) +    # Splines auf dist_to_goal
    s(1) +    # Splines auf angle
    s(2) +    # Splines auf minute
    f(3)      # Faktor für is_standard
).fit(X, y)

In [ ]:
p_goal = gam.predict_proba(X)
shots_df["p_goal_gam"] = p_goal

In [ ]:
auc = roc_auc_score(y, p_goal)
brier = brier_score_loss(y, p_goal)
ll = log_loss(y, p_goal)

print("AUC:", auc)
print("Brier:", brier)
print("Log Loss:", ll)

------------------------------------------------------------

sind die aktionen immer von links nach rechts?\
Team A und Team B in einem Spiel die Schüssen anschauen. wo starte die aktion und wo endet sie für das jeweilige team

In [ ]:
all_actions.head()

In [ ]:
shots = all_actions[all_actions['type_name'] == 'shot']

In [ ]:
# Endpositionen extrahieren
shot_end_positions = shots[['game_id','team_id', 'period_id','time_seconds','end_x', 'end_y','result_id']]


In [ ]:
shot_end_positions[shot_end_positions['game_id'] == 3890561]


---------------------------------------------

Winkel der Schüsse auf das Tor

In [ ]:
# Nur Schüsse betrachten
shots = all_actions[all_actions['type_name'] == 'shot'].copy()

In [ ]:
# Angenommen: Spielfeld geht von x=0 bis x=105 (Opta Standard)
def detect_goal_side(end_x, field_length=105):
    return 'left' if end_x < field_length / 2 else 'right'

def calculate_centered_shot_angle(row):
    # Ziel-Torposition
    if row['goal_side'] == 'right':
        goal_x, goal_y = 105, 34
        dx = goal_x - row['start_x']
    else:
        goal_x, goal_y = 0, 34
        dx = row['start_x'] - goal_x  # Richtung umdrehen!

    dy = goal_y - row['start_y']

    # Jetzt ist dx immer positiv → Richtung zum Tor ist einheitlich
    angle_rad = np.arctan2(dy, dx)
    angle_deg = np.degrees(angle_rad)

    return angle_deg


In [ ]:
# Schritt 1: Torseite erkennen
shots['goal_side'] = shots['start_x'].apply(detect_goal_side)

# Schritt 2: Winkel berechnen
shots['shot_angle_centered'] = shots.apply(calculate_centered_shot_angle, axis=1)


In [ ]:
# Ergebnis
print(shots[['start_x', 'start_y', 'goal_side', 'shot_angle_centered']].head())

Berechnete Winkel in die all_actions aufnehmen


In [ ]:
all_actions['shot_angle_centered'] = np.nan

In [ ]:
# Torseite erkennen (für alle Zeilen, nicht nur Schüsse)
all_actions['goal_side'] = all_actions['start_x'].apply(detect_goal_side)

In [ ]:
# Winkel nur für Schüsse berechnen
shot_mask = all_actions['type_name'] == 'shot'
all_actions.loc[shot_mask, 'shot_angle_centered'] = all_actions[shot_mask].apply(calculate_centered_shot_angle, axis=1)

In [ ]:
all_actions.head()